# Module 10: Morphological Operations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/10_Module.ipynb)

**Week 15: Erosion, Dilation, Opening, Closing, Morphological Gradients**

## Learning Objectives
- Understand morphological operations theory
- Apply erosion and dilation transformations
- Use opening and closing for noise removal
- Extract morphological gradients and boundaries
- Apply top-hat and black-hat transforms
- Solve real-world problems with morphology

---
## ⚠️ IMPORTANT: Run All Cells in Order

**This notebook must be executed sequentially from top to bottom.**

- Click **Runtime → Run all** (or **Cell → Run All**)
- Do NOT skip cells or run them out of order
- Each cell depends on variables from previous cells

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Morphological Operations

### What is Mathematical Morphology?

**Mathematical morphology** is a theory for analyzing spatial structures based on set theory, lattice theory, and topology.

**Key concepts**:
- Operates on **binary images** (foreground/background)
- Uses a **structuring element** (kernel) as a probe
- Based on **Minkowski addition and subtraction**

### Structuring Element

A **structuring element** $B$ is a small binary image (kernel) that defines the neighborhood:

Common shapes:
- **Rectangle**: `cv2.MORPH_RECT`
- **Ellipse**: `cv2.MORPH_ELLIPSE`
- **Cross**: `cv2.MORPH_CROSS`

### Why Morphological Operations?

1. **Noise removal**: Remove small artifacts
2. **Shape analysis**: Extract skeleton, boundary
3. **Segmentation**: Separate touching objects
4. **Feature extraction**: Detect edges, corners
5. **Image enhancement**: Fill holes, smooth boundaries

In [ ]:
# Load and prepare test image
import urllib.request
from urllib.request import Request, urlopen

# Load an image
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/320px-Cat03.jpg'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(req) as response:
    image_data = response.read()
with open('test.jpg', 'wb') as f:
    f.write(image_data)

img = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)

# Create a binary image using thresholding
_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# Create a simple test image with shapes
test_img = np.zeros((200, 200), dtype=np.uint8)
cv2.rectangle(test_img, (50, 50), (150, 150), 255, -1)
cv2.circle(test_img, (100, 100), 30, 0, -1)

# Add some noise
noisy = test_img.copy()
noise_points = np.random.randint(0, 200, (50, 2))
for point in noise_points:
    cv2.circle(noisy, tuple(point), 2, 255, -1)

print(f"Image loaded: {img.shape}")
print(f"Test image created: {test_img.shape}")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(test_img, cmap='gray')
axes[1].set_title('Test Image (Clean)')
axes[1].axis('off')

axes[2].imshow(noisy, cmap='gray')
axes[2].set_title('Test Image (Noisy)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

### 1. Erosion

**Erosion** shrinks the foreground by removing pixels at object boundaries.

#### Mathematical Definition

For image $A$ and structuring element $B$:
$$
A \ominus B = \{z \mid B_z \subseteq A\}
$$

Where $B_z$ is $B$ translated by $z$.

**In words**: Keep pixel if **all** pixels under structuring element are foreground.

#### Properties

1. **Shrinks** objects
2. **Removes** small white objects (noise)
3. **Breaks** thin connections
4. **Enlarges** holes inside objects
5. **Not invertible**: Information is lost

#### Applications

- Remove small noise particles
- Separate touching objects
- Thin foreground regions

In [ ]:
# Erosion demonstration
print("=" * 70)
print("EROSION")
print("=" * 70)

# Create structuring elements
kernel_3x3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
kernel_5x5 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
kernel_cross = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))

print(f"\nStructuring elements created:")
print(f"  3×3 Rectangle")
print(f"  5×5 Rectangle")
print(f"  5×5 Ellipse")
print(f"  5×5 Cross")

# Apply erosion with different kernels
eroded_3x3 = cv2.erode(noisy, kernel_3x3, iterations=1)
eroded_5x5 = cv2.erode(noisy, kernel_5x5, iterations=1)
eroded_2iter = cv2.erode(noisy, kernel_3x3, iterations=2)
eroded_ellipse = cv2.erode(noisy, kernel_ellipse, iterations=1)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(noisy, cmap='gray')
axes[0, 0].set_title('Original (Noisy)')
axes[0, 0].axis('off')

axes[0, 1].imshow(eroded_3x3, cmap='gray')
axes[0, 1].set_title('Eroded (3×3, 1 iter)\nRemoves small noise')
axes[0, 1].axis('off')

axes[0, 2].imshow(eroded_5x5, cmap='gray')
axes[0, 2].set_title('Eroded (5×5, 1 iter)\nMore shrinking')
axes[0, 2].axis('off')

axes[1, 0].imshow(eroded_2iter, cmap='gray')
axes[1, 0].set_title('Eroded (3×3, 2 iter)\nEven more shrinking')
axes[1, 0].axis('off')

axes[1, 1].imshow(eroded_ellipse, cmap='gray')
axes[1, 1].set_title('Eroded (Ellipse)\nRounder shapes')
axes[1, 1].axis('off')

# Show kernels
kernel_display = np.zeros((15, 15), dtype=np.uint8)
kernel_display[5:10, 5:10] = kernel_ellipse * 255
axes[1, 2].imshow(kernel_display, cmap='gray')
axes[1, 2].set_title('Ellipse Kernel (5×5)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Erosion removes small objects and shrinks foreground!")

### 2. Dilation

**Dilation** grows the foreground by adding pixels at object boundaries.

#### Mathematical Definition

$$
A \oplus B = \{z \mid (\hat{B})_z \cap A \neq \emptyset\}
$$

Where $\hat{B}$ is the reflection of $B$.

**In words**: Set pixel to foreground if **any** pixel under structuring element is foreground.

#### Properties

1. **Expands** objects
2. **Fills** small holes and gaps
3. **Connects** nearby objects
4. **Thickens** foreground regions
5. **Not invertible**: Information is lost

#### Duality with Erosion

$$
(A \oplus B)^c = A^c \ominus \hat{B}
$$

Dilating foreground = Eroding background

In [ ]:
# Dilation demonstration
print("=" * 70)
print("DILATION")
print("=" * 70)

# Create image with gaps
gaps_img = np.zeros((200, 200), dtype=np.uint8)
cv2.rectangle(gaps_img, (50, 50), (150, 70), 255, -1)
cv2.rectangle(gaps_img, (50, 80), (150, 100), 255, -1)
cv2.rectangle(gaps_img, (50, 110), (150, 130), 255, -1)

# Apply dilation
dilated_3x3 = cv2.dilate(gaps_img, kernel_3x3, iterations=1)
dilated_5x5 = cv2.dilate(gaps_img, kernel_5x5, iterations=1)
dilated_2iter = cv2.dilate(gaps_img, kernel_3x3, iterations=3)

# Compare erosion and dilation on same image
eroded_test = cv2.erode(test_img, kernel_5x5, iterations=1)
dilated_test = cv2.dilate(test_img, kernel_5x5, iterations=1)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(gaps_img, cmap='gray')
axes[0, 0].set_title('Original (Gaps)')
axes[0, 0].axis('off')

axes[0, 1].imshow(dilated_3x3, cmap='gray')
axes[0, 1].set_title('Dilated (3×3, 1 iter)\nStarts filling gaps')
axes[0, 1].axis('off')

axes[0, 2].imshow(dilated_5x5, cmap='gray')
axes[0, 2].set_title('Dilated (5×5, 1 iter)\nMore filling')
axes[0, 2].axis('off')

axes[1, 0].imshow(test_img, cmap='gray')
axes[1, 0].set_title('Test Image')
axes[1, 0].axis('off')

axes[1, 1].imshow(eroded_test, cmap='gray')
axes[1, 1].set_title('Erosion\nShrinks objects')
axes[1, 1].axis('off')

axes[1, 2].imshow(dilated_test, cmap='gray')
axes[1, 2].set_title('Dilation\nExpands objects')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Dilation fills gaps and expands foreground!")

### 3. Opening

**Opening** = Erosion followed by Dilation

#### Mathematical Definition

$$
A \circ B = (A \ominus B) \oplus B
$$

#### Properties

1. **Removes small objects** (noise)
2. **Smooths** object boundaries
3. **Breaks** thin connections
4. **Preserves** shape and size of large objects
5. **Idempotent**: $(A \circ B) \circ B = A \circ B$

#### Use Cases

- Remove salt noise (white specs)
- Separate touching objects
- Smooth boundaries without changing size much

In [ ]:
# Opening demonstration
print("=" * 70)
print("OPENING (Erosion → Dilation)")
print("=" * 70)

# Apply opening
opened_3x3 = cv2.morphologyEx(noisy, cv2.MORPH_OPEN, kernel_3x3)
opened_5x5 = cv2.morphologyEx(noisy, cv2.MORPH_OPEN, kernel_5x5)

# Manual opening (erosion then dilation)
manual_eroded = cv2.erode(noisy, kernel_3x3, iterations=1)
manual_opened = cv2.dilate(manual_eroded, kernel_3x3, iterations=1)

# Verify they're the same
same = np.allclose(opened_3x3, manual_opened)
print(f"\nManual opening matches cv2.morphologyEx: {same}")

# Visualize step by step
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(noisy, cmap='gray')
axes[0, 0].set_title('Original (Noisy)')
axes[0, 0].axis('off')

axes[0, 1].imshow(manual_eroded, cmap='gray')
axes[0, 1].set_title('Step 1: Erosion\nRemoves noise')
axes[0, 1].axis('off')

axes[0, 2].imshow(manual_opened, cmap='gray')
axes[0, 2].set_title('Step 2: Dilation\nRestores size')
axes[0, 2].axis('off')

axes[1, 0].imshow(opened_3x3, cmap='gray')
axes[1, 0].set_title('Opening (3×3)\nNoise removed')
axes[1, 0].axis('off')

axes[1, 1].imshow(opened_5x5, cmap='gray')
axes[1, 1].set_title('Opening (5×5)\nMore aggressive')
axes[1, 1].axis('off')

# Compare before and after
comparison = np.hstack([noisy, opened_3x3])
axes[1, 2].imshow(comparison, cmap='gray')
axes[1, 2].set_title('Before (left) vs After (right)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Opening removes noise while preserving object size!")

### 4. Closing

**Closing** = Dilation followed by Erosion

#### Mathematical Definition

$$
A \bullet B = (A \oplus B) \ominus B
$$

#### Properties

1. **Fills small holes** and gaps
2. **Connects** nearby objects
3. **Smooths** boundaries
4. **Preserves** shape and size of large objects
5. **Idempotent**: $(A \bullet B) \bullet B = A \bullet B$

#### Duality with Opening

$$
(A \bullet B)^c = A^c \circ \hat{B}
$$

Closing foreground = Opening background

#### Use Cases

- Remove pepper noise (black specs)
- Fill small holes in objects
- Connect broken text or lines

In [ ]:
# Closing demonstration
print("=" * 70)
print("CLOSING (Dilation → Erosion)")
print("=" * 70)

# Create image with holes
holes_img = test_img.copy()
# Add small black holes
hole_points = [(70, 70), (90, 90), (110, 110), (130, 130)]
for point in hole_points:
    cv2.circle(holes_img, point, 3, 0, -1)

# Apply closing
closed_3x3 = cv2.morphologyEx(holes_img, cv2.MORPH_CLOSE, kernel_3x3)
closed_5x5 = cv2.morphologyEx(holes_img, cv2.MORPH_CLOSE, kernel_5x5)

# Manual closing (dilation then erosion)
manual_dilated = cv2.dilate(holes_img, kernel_3x3, iterations=1)
manual_closed = cv2.erode(manual_dilated, kernel_3x3, iterations=1)

# Compare opening and closing on noisy image
opened_noisy = cv2.morphologyEx(noisy, cv2.MORPH_OPEN, kernel_3x3)
closed_noisy = cv2.morphologyEx(noisy, cv2.MORPH_CLOSE, kernel_3x3)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(holes_img, cmap='gray')
axes[0, 0].set_title('Original (Holes)')
axes[0, 0].axis('off')

axes[0, 1].imshow(manual_dilated, cmap='gray')
axes[0, 1].set_title('Step 1: Dilation\nFills holes')
axes[0, 1].axis('off')

axes[0, 2].imshow(manual_closed, cmap='gray')
axes[0, 2].set_title('Step 2: Erosion\nRestores size')
axes[0, 2].axis('off')

axes[1, 0].imshow(noisy, cmap='gray')
axes[1, 0].set_title('Noisy Image')
axes[1, 0].axis('off')

axes[1, 1].imshow(opened_noisy, cmap='gray')
axes[1, 1].set_title('Opening\nRemoves white noise')
axes[1, 1].axis('off')

axes[1, 2].imshow(closed_noisy, cmap='gray')
axes[1, 2].set_title('Closing\nRemoves black noise')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Closing fills holes while preserving object size!")
print("Opening removes white noise, Closing removes black noise.")

### 5. Morphological Gradient

The **morphological gradient** highlights object boundaries.

#### Definitions

**Basic gradient**:
$$
\text{Gradient}(A) = (A \oplus B) - (A \ominus B)
$$

**External gradient** (only external boundary):
$$
\text{External}(A) = (A \oplus B) - A
$$

**Internal gradient** (only internal boundary):
$$
\text{Internal}(A) = A - (A \ominus B)
$$

#### Properties

- Extracts **edges** of objects
- More robust to noise than Sobel/Canny on binary images
- Width depends on structuring element size

In [ ]:
# Morphological gradient demonstration
print("=" * 70)
print("MORPHOLOGICAL GRADIENT")
print("=" * 70)

# Apply morphological gradient
gradient = cv2.morphologyEx(test_img, cv2.MORPH_GRADIENT, kernel_3x3)
gradient_5x5 = cv2.morphologyEx(test_img, cv2.MORPH_GRADIENT, kernel_5x5)

# Manual gradient
dilated = cv2.dilate(test_img, kernel_3x3)
eroded = cv2.erode(test_img, kernel_3x3)
manual_gradient = cv2.subtract(dilated, eroded)

# External and internal gradients
external = cv2.subtract(dilated, test_img)
internal = cv2.subtract(test_img, eroded)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(test_img, cmap='gray')
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(dilated, cmap='gray')
axes[0, 1].set_title('Dilation')
axes[0, 1].axis('off')

axes[0, 2].imshow(eroded, cmap='gray')
axes[0, 2].set_title('Erosion')
axes[0, 2].axis('off')

axes[1, 0].imshow(gradient, cmap='gray')
axes[1, 0].set_title('Morphological Gradient\n(Dilation - Erosion)')
axes[1, 0].axis('off')

axes[1, 1].imshow(external, cmap='gray')
axes[1, 1].set_title('External Gradient\n(Dilation - Original)')
axes[1, 1].axis('off')

axes[1, 2].imshow(internal, cmap='gray')
axes[1, 2].set_title('Internal Gradient\n(Original - Erosion)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Morphological gradient extracts object boundaries!")

### 6. Top-Hat and Black-Hat Transforms

#### Top-Hat (White Top-Hat)

Extracts **small bright objects** or details:
$$
\text{TopHat}(A) = A - (A \circ B)
$$

**Use**: Extract bright features smaller than structuring element

#### Black-Hat

Extracts **small dark objects** or details:
$$
\text{BlackHat}(A) = (A \bullet B) - A
$$

**Use**: Extract dark features smaller than structuring element

#### Applications

- Correct uneven illumination
- Extract text from images
- Enhance small features

In [ ]:
# Top-hat and Black-hat demonstration
print("=" * 70)
print("TOP-HAT AND BLACK-HAT TRANSFORMS")
print("=" * 70)

# Create test image with bright and dark spots
spots_img = np.ones((200, 200), dtype=np.uint8) * 128
# Large background rectangle
cv2.rectangle(spots_img, (40, 40), (160, 160), 200, -1)
# Small bright spots
cv2.circle(spots_img, (60, 60), 5, 255, -1)
cv2.circle(spots_img, (140, 60), 5, 255, -1)
# Small dark spots
cv2.circle(spots_img, (60, 140), 5, 50, -1)
cv2.circle(spots_img, (140, 140), 5, 50, -1)

# Apply top-hat and black-hat
kernel_large = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
tophat = cv2.morphologyEx(spots_img, cv2.MORPH_TOPHAT, kernel_large)
blackhat = cv2.morphologyEx(spots_img, cv2.MORPH_BLACKHAT, kernel_large)

# Manual calculation
opened = cv2.morphologyEx(spots_img, cv2.MORPH_OPEN, kernel_large)
manual_tophat = cv2.subtract(spots_img, opened)

closed = cv2.morphologyEx(spots_img, cv2.MORPH_CLOSE, kernel_large)
manual_blackhat = cv2.subtract(closed, spots_img)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(spots_img, cmap='gray')
axes[0, 0].set_title('Original\n(Bright + Dark spots)')
axes[0, 0].axis('off')

axes[0, 1].imshow(opened, cmap='gray')
axes[0, 1].set_title('Opening\nRemoves bright spots')
axes[0, 1].axis('off')

axes[0, 2].imshow(tophat, cmap='gray')
axes[0, 2].set_title('Top-Hat\nOriginal - Opening\nExtracts bright spots')
axes[0, 2].axis('off')

axes[1, 0].imshow(spots_img, cmap='gray')
axes[1, 0].set_title('Original')
axes[1, 0].axis('off')

axes[1, 1].imshow(closed, cmap='gray')
axes[1, 1].set_title('Closing\nFills dark spots')
axes[1, 1].axis('off')

axes[1, 2].imshow(blackhat, cmap='gray')
axes[1, 2].set_title('Black-Hat\nClosing - Original\nExtracts dark spots')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("  Top-Hat extracts bright features smaller than kernel")
print("  Black-Hat extracts dark features smaller than kernel")

## Summary

### Key Morphological Operations

| Operation | Formula | Effect | Use Case |
|-----------|---------|--------|----------|
| **Erosion** | $A \ominus B$ | Shrinks objects | Remove noise |
| **Dilation** | $A \oplus B$ | Expands objects | Fill gaps |
| **Opening** | $(A \ominus B) \oplus B$ | Remove small objects | Denoise (white) |
| **Closing** | $(A \oplus B) \ominus B$ | Fill small holes | Denoise (black) |
| **Gradient** | $(A \oplus B) - (A \ominus B)$ | Extract boundaries | Edge detection |
| **Top-Hat** | $A - (A \circ B)$ | Extract bright spots | Feature extraction |
| **Black-Hat** | $(A \bullet B) - A$ | Extract dark spots | Feature extraction |

### Choosing the Right Operation

**Problem: Remove white noise** → Use Opening  
**Problem: Remove black noise** → Use Closing  
**Problem: Fill small holes** → Use Closing  
**Problem: Separate touching objects** → Use Opening or Erosion  
**Problem: Extract edges** → Use Morphological Gradient  
**Problem: Enhance small bright features** → Use Top-Hat  
**Problem: Enhance small dark features** → Use Black-Hat  

### OpenCV Functions

| Operation | Function | Parameters |
|-----------|----------|------------|
| Erosion | `cv2.erode(img, kernel, iterations)` | kernel, iterations |
| Dilation | `cv2.dilate(img, kernel, iterations)` | kernel, iterations |
| Opening | `cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)` | cv2.MORPH_OPEN |
| Closing | `cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)` | cv2.MORPH_CLOSE |
| Gradient | `cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)` | cv2.MORPH_GRADIENT |
| Top-Hat | `cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)` | cv2.MORPH_TOPHAT |
| Black-Hat | `cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)` | cv2.MORPH_BLACKHAT |

### Structuring Elements

```python
# Rectangle
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# Ellipse (circular)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# Cross
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))
```

**Next**: Module 11 - Advanced Topics